In [2]:
import numpy as np
import sklearn 
import subprocess
import re 
from io import StringIO 
from datetime import date, timedelta, datetime 
import sys
import os
import time
import boto3
import pandas as pd
from multiprocessing import Pool
import multiprocessing
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

!conda install -c conda-forge pygrib -n python3 -y
import pygrib

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - pygrib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libnetcdf-4.6.1            |       h5e45101_3         1.3 MB  conda-forge
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    pygrib-2.0.3               |   py36h5688137_0         1.2 MB  conda-forge
    hdf4-4.2.13                |                0         969 KB  conda-forge
    eccodes-2.8.2              |       ha8b302a_0         5.6 MB  conda-forge
    proj4-4.9.3                |       h470a237_8         3.2 MB  conda-forge
    pyproj-1.9.5.1             |   py36h508ed2a_5         134 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        15.4 MB

The following NEW packages w

In [2]:
hours = np.arange(24)
fxx = [0]
field = 'prs'
model = 'hrrr'
sDATE = date(2017, 1, 1)          # Start date
eDATE = date(2017, 12, 31)          # End date (exclusive)
days = (eDATE-sDATE).days
DATES = [sDATE + timedelta(days=d) for d in range(days)]

d = DATES[0]

# for h in np.arange(10):
#     path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, h, field, fxx[0])
#     bucket_name = 'jedi-pub'
#     key_data = 'remi/' + path_grb2
#     s3 = boto3.resource('s3')
#     s3.Bucket(bucket_name).download_file(key_data, path_grb2)

In [3]:
#Function which return a matrix with all date from an hour
def one_hour_mat(d, model, hour, field, forecast):
    var_name = ['']*686
    mat_res = np.zeros((1905141,686))
    path_grb2 = "%s_%s.t%02dz.wrf%sf%02d.grib2" % (d.strftime('%Y%m%d'), model, hour, field, forecast)
    grbs = pygrib.open(path_grb2)
    for j in range(684):
        grb = grbs.readline()
        var_name[j] = grb.name
        mat_res[:,j] =  grb.values.flatten()
    lats, lons = grb.latlons()    
    mat_res[:,684] = lats.flatten()
    mat_res[:,685] = lons.flatten()
    grbs.close()
    mat_res[np.where(mat_res==9.969209968386869e+36)] = 0
    return (mat_res, np.array(var_name))

#Function to scale matrices
def scale_matrices(X_train, y_train, X_test, y_test, X_valid=None, y_valid=None):
    y_train = np.reshape(y_train, (-1,1))
    y_test = np.reshape(y_test, (-1,1))
    if (y_valid is not None):
        y_valid = np.reshape(y_valid, (-1,1))
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    if (X_valid is not None):
        X_valid = scaler.transform(X_valid)
    scaler.fit(y_train)
    y_train = scaler.transform(y_train)
    y_test = scaler.transform(y_test)
    if (y_valid is not None):
        y_valid = scaler.transform(y_valid)
    if (y_valid is not None):      
        return(X_train, X_test, y_train, y_test, X_valid, y_valid, scaler)    
    else:
        return(X_train, X_test, y_train, y_test, scaler)

#Function to print the n most important variables in a linear regresssion
def linreg_mostvar(reglin_coeffs,var_name, n):
    var = ['']*n
    for i in range(n):
        var[i] =  var_name[np.where(reglin_coeffs==np.sort(reglin_coeffs)[0,i] )[1][0]  ]
    return(var)    

In [4]:
#Get the full data matrix
#start = time.time()
# start = time.time()
mat_tot, var_name = one_hour_mat(d, model, 0, field, fxx[0])
# print(mat_tot.shape)
# for h in np.arange(1,3):
#     mat_tot = np.append(mat_tot, one_hour_mat(d, model, h, field, fxx[0])[0], axis=0)    
# end = time.time()
# print(end-start)
print(mat_tot.shape)

(1905141, 686)


In [6]:
#Prepare matrices
Y = mat_tot[:,np.shape(mat_tot)[1]-6]
indices = np.array([i for i in range(np.shape(mat_tot)[1]-6)])
indices = np.append(indices,[np.shape(mat_tot)[1]-2,np.shape(mat_tot)[1]-1])
X = mat_tot[:,indices]
del(mat_tot)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test, scaler = scale_matrices(X_train, y_train, X_test, y_test)

del(X)
del(Y)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1524112, 682)
(1524112, 1)
(381029, 682)
(381029, 1)


In [7]:
#Perform a linear regression
start = time.time()
reg = LinearRegression().fit(X_train, y_train)
end = time.time()
print(end-start)

66.92035937309265


In [8]:
print(reg.score(X_train,y_train))
y_predict = reg.predict(X_test)
print(mean_squared_error(y_test, y_predict))
print(mean_squared_error(scaler.inverse_transform(y_test), scaler.inverse_transform(y_predict)))

0.980995623583358


0.01883458354013145